In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps


In [ ]:
taxi_zones = "../../../Desktop/NYC Taxi & FHV Data/Taxi zones

In [5]:
clean_HigVol_june_2019 = "../../../Desktop/NYC Taxi & FHV Data/clean_csvs/HighVol_june_2019.csv"
clean_hvfhv_june_2019_df = pd.read_csv(clean_HigVol_june_2019)
clean_hvfhv_june_2019_df.head()

,Unnamed: 0,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,pickup_date,pickup_time,dropoff_date,dropoff_time
0,90,HV0004,B02800,2019-06-01 00:09:57,2019-06-01 00:16:41,237,263,2019-06-01,00:09:57,2019-06-01,00:16:41
1,143,HV0003,B02869,2019-06-01 00:20:37,2019-06-01 00:37:38,45,263,2019-06-01,00:20:37,2019-06-01,00:37:38
2,171,HV0003,B02884,2019-06-01 00:30:42,2019-06-01 00:41:14,170,263,2019-06-01,00:30:42,2019-06-01,00:41:14
3,199,HV0003,B02764,2019-06-01 00:34:54,2019-06-01 00:39:28,263,236,2019-06-01,00:34:54,2019-06-01,00:39:28
4,273,HV0003,B02835,2019-06-01 00:34:46,2019-06-01 01:18:08,179,263,2019-06-01,00:34:46,2019-06-01,01:18:08


In [ ]:
clean_HigVol_june_2019 = "../../../Desktop/NYC Taxi & FHV Data/clean_csvs/HighVol_june_2019.csv"
clean_hvfhv_june_2019_df = pd.read_csv(clean_HigVol_june_2019)
clean_hvfhv_june_2019_df.head()